## Importing Libraries

In [1]:
import torch                                                                    
import torchvision                                                              
import torchvision.transforms as transforms    
import numpy as np
import random

## Checking if GPU is available or not

In [2]:
torch.cuda.is_available()

True

## Loading Datasets

In [3]:
from torchvision import datasets                                                # Access to benchmarking datasets

train_set = datasets.MNIST('./data_MNIST', train=True, download=True,                 # Downloading the training dataset in ./data folder
                    transform=transforms.Compose([                              # and transforming the images to tensors and normalizing with
                        transforms.ToTensor(),                                  # mean and std of the entire dataset
                        transforms.Normalize((0.1307,), (0.3081,))
                    ]))

test_set = datasets.MNIST('./data_MNIST', train=False, download=True,                 # Downloading the test dataset in ./data folder 
                    transform=transforms.Compose([                              # and transforming the images to tensors and normalizing with
                        transforms.ToTensor(),                                  # mean and std of the entire dataset
                        transforms.Normalize((0.1307,), (0.3081,))
                    ]))

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:48<00:00, 202898.69it/s]


Extracting ./data_MNIST\MNIST\raw\train-images-idx3-ubyte.gz to ./data_MNIST\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 137202.52it/s]


Extracting ./data_MNIST\MNIST\raw\train-labels-idx1-ubyte.gz to ./data_MNIST\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:02<00:00, 732471.53it/s]


Extracting ./data_MNIST\MNIST\raw\t10k-images-idx3-ubyte.gz to ./data_MNIST\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<?, ?it/s]

Extracting ./data_MNIST\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./data_MNIST\MNIST\raw



In [4]:
def num_to_categorical(y, num_classes):
    """ 1-hot encodes a tensor """
    return np.eye(num_classes, dtype='uint8')[y]

def categorical_to_num(y):
  """ Decodes a 1-hot encoding """
  return np.argmax(y)

In [5]:
# Dataset is there to be able to interact with DataLoader
from torch.utils.data import Dataset                                            

class mnist_data(Dataset):
  '''
  mnist_data class to create an iteratable on our custom mnist dataset:
  where data = (image and number label as one-hot-encoding) and labels = (image)

  Attributes:
    mnist: mnist data object
  Methods:
    len: returns length of the dataset 
    getitem: custom getitem method returns data and labels
  '''

  def __init__(self, d):
    self.mnist = d                                                              # Initialize with mnist data object
    
  def __getitem__(self, index):
                                                     
    image, label = self.mnist[index]   
    number_encoding = num_to_categorical(label, 10)                             # 1-hot encoding of number 
    number_encoding = torch.tensor(number_encoding)

    return (image.flatten(), number_encoding), image                                     # (input), (output) to network      

  def __len__(self):                                                            # Return the length of the dataset
    return len(self.mnist)

In [6]:
train_set_custom = mnist_data(train_set)                                        # custom train_set from mnist_data class
test_set_custom = mnist_data(test_set)       

In [7]:
use_cuda = torch.cuda.is_available()                                            # use gpu if available
device = torch.device("cuda" if use_cuda else "cpu")                            # assigning gpu to variable device

# If you load your samples in the Dataset on CPU and would like to push it during training to the GPU, 
# you can speed up the host to device transfer by enabling pin_memory.
# num_workers attribute tells the data loader instance how many sub-processes to use for data loading
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}             
batch_size = 100                                                                # set size of batch                                    

train_loader = torch.utils.data.DataLoader(                                     # For iterating through our dataset in batches      
    train_set_custom,
    batch_size = batch_size,  
    shuffle=True,                                                               # Shuffle the data points randomly
    **kwargs
)

test_loader = torch.utils.data.DataLoader(
    test_set_custom,
    batch_size = batch_size,
    shuffle=True, 
    **kwargs
)

## VAE Network Architecture

In [8]:
import torch; torch.manual_seed(0)
import torch.nn as nn
import torch.nn.functional as F
import torch.utils
import torch.distributions
import matplotlib.pyplot as plt
import torch.optim as optim

In [9]:
class VariationalEncoder(nn.Module):
    def __init__(self, latent_dims):
        super(VariationalEncoder, self).__init__()
        self.linear1 = nn.Linear(784, 512)
        self.linear2 = nn.Linear(512, latent_dims)
        self.linear3 = nn.Linear(512, latent_dims)

        self.N = torch.distributions.Normal(0, 1)
        self.N.loc = self.N.loc.cuda() # hack to get sampling on the GPU
        self.N.scale = self.N.scale.cuda()
        self.kl = 0

    def forward(self, x, num):
        x = F.relu(self.linear1(x))
        mu =  self.linear2(x)
        sigma = torch.exp(self.linear3(x))
        z = mu + sigma*self.N.sample(mu.shape)

        ## Adding the one-hot encoding of the number 
        z = z+num                                    

        self.kl = (sigma**2 + mu**2 - torch.log(sigma) - 1/2).sum()
        return z

In [10]:
class Decoder(nn.Module):
    def __init__(self, latent_dims):
        super(Decoder, self).__init__()
        self.linear1 = nn.Linear(latent_dims, 512)
        self.linear2 = nn.Linear(512, 784)

    def forward(self, z):
        z = F.relu(self.linear1(z))
        z = torch.sigmoid(self.linear2(z))
        return z.reshape((-1, 1, 28, 28))

In [11]:
class VariationalAutoencoder(nn.Module):
    def __init__(self, latent_dims):
        super(VariationalAutoencoder, self).__init__()
        self.encoder = VariationalEncoder(latent_dims)
        self.decoder = Decoder(latent_dims)

    def forward(self, x1, x2):
        z = self.encoder(x1, x2)
        return self.decoder(z)

In [12]:
latent_dims = 10
device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = VariationalAutoencoder(latent_dims).to(device) 

In [13]:
model

VariationalAutoencoder(
  (encoder): VariationalEncoder(
    (linear1): Linear(in_features=784, out_features=512, bias=True)
    (linear2): Linear(in_features=512, out_features=10, bias=True)
    (linear3): Linear(in_features=512, out_features=10, bias=True)
  )
  (decoder): Decoder(
    (linear1): Linear(in_features=10, out_features=512, bias=True)
    (linear2): Linear(in_features=512, out_features=784, bias=True)
  )
)

In [14]:
train_losses = []
test_losses = []

def train_vae(model, train_data, test_data, epochs=20):
    opt = torch.optim.Adam(model.parameters())
    for epoch in range(epochs):

        model.train()
        for x, y in train_data:
            x1, x2 = x
            x1 = x1.to(device) # GPU
            x2 = x2.to(device) # GPU
            y = y.to(device) # GPU
            opt.zero_grad()
            x_hat = model(x1, x2)
            train_loss = ((y - x_hat)**2).sum() + model.encoder.kl
            train_loss.backward()
            opt.step()

        model.eval()
        test_loss_sum = 0 
        c = 0
        for x, y in test_data:
            x1, x2 = x
            x1 = x1.to(device) # GPU
            x2 = x2.to(device) # GPU
            y = y.to(device) # GPU
            x_hat = model(x1, x2)
            test_loss = ((y - x_hat)**2).sum() + model.encoder.kl
            test_loss_sum += test_loss
            c += 1

        test_loss_avg = test_loss_sum/c
        print(f'Epoch {epoch}. Train loss: {train_loss}, Test loss: {test_loss_avg}')

        train_losses.append(train_loss)
        test_losses.append(test_loss_avg)
        
    return model

model = train_vae(model, train_loader, test_loader, 50)

RuntimeError: DataLoader worker (pid(s) 26492) exited unexpectedly

In [ ]:
torch.save(model.state_dict(), 'MNIST.pth')